In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 8
block_size = 4
max_iters = 1000
learning_rate = 2e-5
eval_iters = 100
n_embed = 384
n_head = 4
n_layer = 4
dropout = 0.2

cuda


In [6]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('trill', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        # input og size (batch, time-step , channels)
        # output of size (batch, time-step, head size)
        
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        
        # compute attention scores ("affinities")
        
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5  # (B,T,hs) @ (B,hs,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)        
        wei = F.softmax(wei,dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B,T,T) @ (B,T,hs) -> (B,T,hs)
        return out
        

In [ ]:
class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''
    
    def __init__(self, num_heads,head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads,n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        # (B,T,F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embed) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed,n_embed),
            nn.Dropout(dropout)
        )
    
    def forward(self,x):
        return self.net(x)  

In [7]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation"""
    
    def __init__(self, n_embed,n_head) -> None:
        super().__init__()
        head_size = n_embed// n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

In [8]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        
        self.blocks = nn.Sequential(*[Block(n_embed,n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed,vocab_size)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight , mean= 0.0, std= 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.weight, mean=0.0, std=0.02)
        
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
    
    def forward(self,index,targets = None):
        B,T = index.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        
        x = tok_emb + pos_emb 
        
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self, index,max_new_tokens):
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            index_cond = index[:,-block_size:]
            
            # get the predictions 
            logits, loss = self.forward(index_cond)
            
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            
            # applying softmax to get probabilities
            probs = F.softmax(logits,dim=-1) # (B,C)
            
            # sample from the distribution
            index_net = torch.multinomial(probs,num_samples=1) # (B,1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_net), dim =1) # (B,T+1)
        
        return index

model = GPTLanguageModel(vocab_size)

m = model.to(device)

NameError: name 'vocab_size' is not defined